In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install vit_pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/87.2 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.7 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/41.6 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 4.3 MB/s eta 0:00:00


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from vit_pytorch import ViT

# Set device to use
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define data transformations
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    transforms.Resize(32)
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    transforms.Resize(32)
])

# Load CIFAR10 dataset
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

# Define data loaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)


# define model
model = ViT(
    image_size = 32,
    patch_size = 4,
    num_classes = 10,
    dim = 64,
    depth = 6,
    heads = 8,
    mlp_dim = 128,
    dropout = 0.1,
    emb_dropout = 0.1
)

# define optimizer and loss function
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

# Train the model
num_epochs = 200
for epoch in range(num_epochs):
    # Train the model
    model.train()
    train_loss = 0
    train_correct = 0
    for images, labels in trainloader:
#         images = images.to(device)
#         labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        train_correct += (outputs.argmax(dim=1) == labels).sum().item()

    # Test the model
    model.eval()
    test_loss = 0
    test_correct = 0
    with torch.no_grad():
        for images, labels in testloader:
#             images = images.to(device)
#             labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            test_loss += loss.item() * images.size(0)
            test_correct += (outputs.argmax(dim=1) == labels).sum().item()

    # Print results for this epoch
    train_loss /= len(trainset)
    train_acc = train_correct / len(trainset)
    test_loss /= len(testset)
    test_acc = test_correct / len(testset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')

# Save the trained model
torch.save(model.state_dict(), 'VIT_cifar10.pth')


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data


Files already downloaded and verified


Epoch [1/200], Train Loss: 1.9365, Train Acc: 0.2649, Test Loss: 1.6988, Test Acc: 0.3578


Epoch [2/200], Train Loss: 1.6570, Train Acc: 0.3889, Test Loss: 1.4528, Test Acc: 0.4726


...

Epoch [64/200], Train Loss: 0.7694, Train Acc: 0.7293, Test Loss: 0.7640, Test Acc: 0.7361


Epoch [65/200], Train Loss: 0.7696, Train Acc: 0.7272, Test Loss: 0.6917, Test Acc: 0.7628


Epoch [66/200], Train Loss: 0.7672, Train Acc: 0.7283, Test Loss: 0.6955, Test Acc: 0.7598


Epoch [67/200], Train Loss: 0.7620, Train Acc: 0.7316, Test Loss: 0.7167, Test Acc: 0.7519


Epoch [68/200], Train Loss: 0.7598, Train Acc: 0.7303, Test Loss: 0.7270, Test Acc: 0.7488
